In [5]:
import os
import pandas as pd

In [6]:
input_path = r""

In [7]:
def find_sample_folders(path):
    paths = []
    valid_paths = []
    for root, dirs, files in os.walk(path):
        for d in dirs:
            paths.append(os.path.join(root, d))
    for p in paths:
        if os.path.isfile(p+ '/X0Y0R1W1C0.tif'): #improve condition
            valid_paths.append(p)
    return valid_paths


def extract_FoV(path):
    """
    get the name of field of views for a sample (format - XnYnRnWnCn)
    para: path - string
    return: fovs - set of string (unique names only)
    """
    fovs = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith('.tif'):
                fovs.append(name[:10])
    fovs = set(fovs)
    return fovs

In [8]:
def coincidence_summary(path):
    """
    This function processes single sample
    para: path - string: sample path
    """
    summary_df = pd.DataFrame()
    fovs = extract_FoV(path)
    for fov in fovs:
        fov_path = path + '/' + fov + '_summary.txt'
        df = pd.read_csv(fov_path, delimiter='\t')
        df['FoV'] = fov
        if os.path.basename(path).startswith('Blank'):
            df['Sample'] = 'Blank'
            df['Rep'] = os.path.basename(path)[-1] + os.path.basename(path)[-8]
        else:
            df['Sample'] = os.path.basename(path)[:-5]
            df['Rep'] = os.path.basename(path)[-1]
        summary_df = pd.concat([summary_df, df])
    return(summary_df)

In [9]:
# Generate summary of results in format of .csv
summary_df = pd.DataFrame()
sample_paths = find_sample_folders(input_path)
for sample_path in sample_paths:
    df = coincidence_summary(sample_path)
    summary_df = pd.concat([summary_df, df])
summary_df = summary_df[['Sample','Rep','FoV','Particles_in_Ch1','Particles_in_Ch2','Colocalized_ch1&ch2','%_Ch1_coloc_ch1&ch2','%_Ch2_coloc_ch1&ch2']]
summary_df = summary_df.fillna(value = 0)
summary_df.to_csv(input_path + '/ResultSummary.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [10]:
def size_intensity_analysis(path):
    """
    This function processes single sample
    para: path - string: sample path
    """
    summary_df = pd.DataFrame()
    fovs = extract_FoV(path)
    for fov in fovs:
        fov_path = path + '/' + fov + '_results.csv'
        try:
            df = pd.read_csv(fov_path)
        except:
            pass
        else:
            df = df.loc[df.Abs_frame == 1]
            df = df[['X_(px)', 'Y_(px)', 'xMin', 'yMin', 'xMax', 'yMax', 'NArea', 'IntegrIntCh1']]
            df['FoV'] = fov
            if os.path.basename(path).startswith('Blank'):
                df['Sample'] = 'Blank'
                df['Rep'] = os.path.basename(path)[-1] + os.path.basename(path)[-8]
            else:
                df['Sample'] = os.path.basename(path)[:-5]
                df['Rep'] = os.path.basename(path)[-1]
            summary_df = pd.concat([summary_df, df])
    return(summary_df)

In [11]:
#Generate amyloid beta species analysis data
summary_df = pd.DataFrame()
sample_paths = find_sample_folders(input_path)
for sample_path in sample_paths:
    df = size_intensity_analysis(sample_path)
    summary_df = pd.concat([summary_df, df])
summary_df = summary_df[['Sample','Rep','FoV','X_(px)', 'Y_(px)', 'xMin', 'yMin', 'xMax', 'yMax', 'NArea', 'IntegrIntCh1']]
summary_df = summary_df.fillna(value = 0)
summary_df.to_csv(input_path + '/AmyloidAnalysis.csv')